In [1]:
using LinearRegression
using Statistics
using CairoMakie
using DelimitedFiles
using Megafauna
using Distances
using HypothesisTests
include("../src/SegmentDistances.jl")
include("../src/DPC.jl")

using OptimalTransport
using ProgressMeter

In [2]:
T = 150000
X_shifted = readdlm("../data/VDP/V_3D_1ps_shifted.dat")[1:T, 2:4] ./ 360
X = (readdlm("../data/VDP/V_3D_1ps.dat")[1:T,2:4] .+ 180) ./ 360
Q = [0.98 0.9 0.97]
W = [100 25 100]
N = [2 7 3]
jd_cps = findall(x -> any(y -> y > 0, x), eachrow(readdlm("../data/VDP/V_3D_1ps_shifted.lam10.0alpha0.7.transitionProba.dat")[1:T,2:4]));

x1_cps, x1_labels, _ = periodic_cluster_1d(X[:,1], Q[1], W[1], N[1])
x2_cps, x2_labels, _ = periodic_cluster_1d(X[:,2], Q[2], W[2], N[2])
x3_cps, x3_labels, _ = periodic_cluster_1d(X[:,3], Q[3], W[3], N[3])

x1_cps = [t ∈ x1_cps ? 1 : 0 for t=1:T]
x2_cps = [t ∈ x2_cps ? 1 : 0 for t=1:T]
x3_cps = [t ∈ x3_cps ? 1 : 0 for t=1:T]

mf_cps = findall(x -> any( y -> y > 0, x), eachrow(cat(x1_cps, x2_cps, x3_cps, dims=2)))

enumerating change points
number of dimensions: 1
Number of segments = 47
Computing 1081 segment distances


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


finished computing distances
9.16328667100115e-9
enumerating change points
number of dimensions: 1
Number of segments = 1007
Computing 506521 segment distances


Progress: 100%|█████████████████████████████████████████| Time: 0:00:33


finished computing distances
1.4586214895741465e-7
enumerating change points
number of dimensions: 1
Number of segments = 151
Computing 11325 segment distances


Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


finished computing distances
5.2318697666942846e-11


1197-element Vector{Int64}:
      1
    376
    384
    494
    499
    547
    556
    567
    570
    608
    611
    614
    622
      ⋮
 148088
 148258
 148259
 148262
 148264
 148308
 148317
 148386
 148393
 148487
 148495
 150000

In [3]:
bart_cps = convert(Array{Int32}, readdlm("../data/VDP/vdp_joint_cps.txt"))
bart_cps = bart_cps[1:findfirst(x -> x > T, bart_cps)[1] - 1]
bart_cps[1] = 1

1

In [4]:
function dists_euc(X, cps)
    S = length(cps) - 1
    println(S)
    D = zeros(S,S)
    d = Euclidean()
    C(X,Y) = pairwise(d, X', Y')
    ε = 0.25
    @showprogress for i=1:S, j=1:S
        if i > j
            t0, t1 = cps[i], cps[i+1]
            s0, s1 = cps[j], cps[j+1]
            Si, Sj = X[t0:t1,:], X[s0:s1,:]
            a, b = fill(1, t1 - t0 + 1) / (t1 - t0 + 1), fill(1, s1 - s0 + 1) / (s1 - s0 + 1)
            D[i,j] = D[j,i] = sqrt(sinkhorn2(a, b, C(Si, Sj), ε))
        end
    end
        return D
end

function dists(X, cps)
    S = length(cps) - 1
    println(S)
    D = zeros(S,S)
    d = PeriodicEuclidean([1.0 1.0 1.0])
    C(X,Y) = pairwise(d, X', Y').^2
    ε = 0.25
    @showprogress for i=1:S, j=1:S
        if i > j
            t0, t1 = cps[i], cps[i+1]
            s0, s1 = cps[j], cps[j+1]
            Si, Sj = X[t0:t1,:], X[s0:s1,:]
            a, b = fill(1, t1 - t0 + 1) / (t1 - t0 + 1), fill(1, s1 - s0 + 1) / (s1 - s0 + 1)
            D[i,j] = D[j,i] = sqrt(sinkhorn2(a, b, C(Si, Sj), ε))
        end
    end
        return D
end

dists (generic function with 1 method)

In [5]:
SIMPLED = dists_euc(X, jd_cps)

3275


Progress: 100%|█████████████████████████████████████████| Time: 0:02:32


3275×3275 Matrix{Float64}:
 0.0       0.370476  0.331675  0.401393  …  0.343763  0.334624  0.368825
 0.370476  0.0       0.370811  0.332755     0.350532  0.391384  0.318353
 0.331675  0.370811  0.0       0.391142     0.325106  0.305224  0.366052
 0.401393  0.332755  0.391142  0.0          0.355206  0.414681  0.310598
 0.382066  0.322906  0.379768  0.300274     0.352474  0.40236   0.312352
 0.332214  0.37858   0.309751  0.399235  …  0.326571  0.300191  0.373653
 0.347845  0.335425  0.339626  0.345845     0.329468  0.352929  0.328592
 0.39119   0.323509  0.394034  0.313622     0.365944  0.417316  0.315943
 0.334938  0.37333   0.314117  0.391537     0.326356  0.30938   0.367722
 0.669909  0.68068   0.688478  0.725247     0.697107  0.69163   0.688965
 0.632716  0.635325  0.65215   0.680444  …  0.657664  0.657865  0.64382
 0.36254   0.344861  0.353336  0.349541     0.340067  0.366676  0.338583
 0.651205  0.708994  0.645774  0.739889     0.672207  0.626115  0.711868
 ⋮                       

In [6]:
BarTD = dists(X, bart_cps)

3899


Progress:  61%|████████████████████████▉                |  ETA: 0:01:26

LoadError: InterruptException:

In [7]:
MFD = dists(X, mf_cps)

1196


Progress:  51%|████████████████████▉                    |  ETA: 0:00:58

LoadError: InterruptException:

In [9]:
SIMPLED = readdlm("/home/dcg/projects/DPA/vdp_seg_simple_dists.txt")
BarTD = readdlm("/home/dcg/projects/DPA/vdp_seg_bart_dists.txt")
MFD = readdlm("/home/dcg/projects/DPA/vdp_seg_mf_dists.txt")

1196×1196 Matrix{Float64}:
 0.0       0.437727  0.457129  0.252255  …  0.342227  0.215945  0.132695
 0.437727  0.0       0.119469  0.367375     0.550749  0.467616  0.438703
 0.457129  0.119469  0.0       0.380034     0.56681   0.486462  0.45691
 0.252255  0.367375  0.380034  0.0          0.411432  0.304372  0.251818
 0.138096  0.444164  0.462172  0.256515     0.345311  0.196333  0.133723
 0.137242  0.448065  0.46693   0.258261  …  0.344407  0.211686  0.132381
 0.292249  0.455572  0.470242  0.341588     0.434887  0.378292  0.290561
 0.470301  0.496914  0.496626  0.47951      0.592686  0.520329  0.474611
 0.438486  0.52192   0.524985  0.463337     0.550563  0.492007  0.441856
 0.506408  0.503791  0.497887  0.497597     0.602789  0.541376  0.511342
 0.522226  0.463664  0.455699  0.504253  …  0.62141   0.567041  0.526229
 0.438146  0.560866  0.567277  0.477919     0.547229  0.469751  0.443917
 0.430271  0.574514  0.575335  0.475325     0.537457  0.477153  0.434087
 ⋮                       

In [10]:
function get_dp_stats(D; q=0.02, seg_lengths=nothing)
    N, _ = size(D)
    dc = quantile!([D[i,j] for i=1:N, j=1:N if i > j], q)
    if isnothing(seg_lengths)
        ρ = hcat(collect(1:N), [length(row[row .<= dc]) for row in eachrow(D)])
    else
        ρ = hcat(collect(1:N), [sum(seg_lengths .* exp.(-(D[i,:] / dc).^2)) for i in 1:N])
    end
    δ = zeros(N)
    nearest_denser_neighbor = zeros(N)
    for i=1:N
        indices = convert(Array{Int32},filter!(idx -> idx ∉ [i], ρ[ρ[:,2] .>= ρ[i, 2],:][:,1]))
        if length(indices) > 0
            δ[i] = minimum(D[indices, i])
            nearest_denser_neighbor[i] = argmin(D[indices, i])
        else
            δ[i] = maximum(D[:,i])
            nearest_denser_neighbor[i] = i
        end
    end
    γ = ρ[:,2] .* δ
    return (ρ, δ, γ)
end

bart_segment_lengths = [bart_cps[T+1] - bart_cps[T] for T in 1:length(bart_cps) - 1]
mf_segment_lengths = [mf_cps[T+1] - mf_cps[T] for T in 1:length(mf_cps) - 1]
jd_segment_lengths = [jd_cps[T+1] - jd_cps[T] for T in 1:length(jd_cps) - 1]

lengths = [jd_segment_lengths, bart_segment_lengths, mf_segment_lengths]

fig = Figure(size=(900, 900))

title = ["SIMPLE", "BarT", "MF"]
dist_mat = [
    SIMPLED,
    BarTD,
    MFD
]

for i=1:3
    ax1 = Axis(fig[i, 1])
    ax2 = Axis(fig[i, 2])
    ax3 = Axis(fig[i, 3])

    ax1.title = "$(title[i]) ρ,δ"
    ax2.title = "$(title[i]) γ"
    ax3.title = "$(title[i]), logγ"
    ρ, δ, γ = get_dp_stats(dist_mat[i], seg_lengths=lengths[i])
    scatter!(ax1, ρ[:,2], δ)
    ax1.xlabel = "ρ"
    ax1.ylabel = "δ" 
    ax2.xlabel = "Rank"
    ax2.ylabel = "γ" 
    ax3.xlabel = "Rank"
    ax3.ylabel = "log γ"
    markersize=5
    scatter!(ax1, ρ[:,2], δ, markersize=markersize)
    scatter!(ax2, 1:length(γ), sort!(γ), markersize=markersize)
    scatter!(ax3, 1:length(γ), log.(γ),markersize=markersize)
end

save("vdp-dc.png", fig)

CairoMakie.Screen{IMAGE}


In [ ]:
open("/home/dcg/projects/DPA/vdp_seg_simple_dists.txt", "w") do io
        writedlm(io, SIMPLED)
end
open("/home/dcg/projects/DPA/vdp_seg_bart_dists.txt", "w") do io
        writedlm(io, BarTD)
end
open("/home/dcg/projects/DPA/vdp_seg_mf_dists.txt", "w") do io
        writedlm(io, MFD)
end

In [11]:
#seg_labels = get_clusters(X, jd_cps, D, 10)
using StatsBase
#simple_seg_labels = convert(Array{Int32}, readdlm("/home/dcg/projects/DPA/vdp_seg_simple_dpa_labels.txt"))[:,1]
#simple_pt_labels = label_series(X, jd_cps, simple_seg_labels)
bart_seg_labels = convert(Array{Int32}, readdlm("/home/dcg/projects/DPA/vdp_seg_bart_dists_dpa_labels.txt"))[:,1]
bart_pt_labels = label_series(X, bart_cps, bart_seg_labels)
mf_seg_labels = convert(Array{Int32}, readdlm("/home/dcg/projects/DPA/vdp_seg_mf_dists_dpa_labels.txt"))[:,1]
mf_pt_labels = label_series(X, mf_cps, mf_seg_labels)

point_labels = [bart_pt_labels, mf_pt_labels]
titles = ["BarT", "MF"]
function relabel_by_frequency(l)
    # Count the frequency of each label
    freq_dict = Dict{eltype(l), Int}()
    for label in l
        freq_dict[label] = get(freq_dict, label, 0) + 1
    end
    
    # Sort labels by their frequency (most frequent first)
    sorted_labels = sort(collect(keys(freq_dict)), by=x->freq_dict[x], rev=true)
    
    # Create mapping from original labels to new labels (0, 1, 2, ...)
    label_map = Dict(label => i-1 for (i, label) in enumerate(sorted_labels))
    
    # Create the new label array
    k = [label_map[label] for label in l]
    
    return k
end
mf_pt_labels = relabel_by_frequency(mf_pt_labels)
bart_pt_labels = relabel_by_frequency(bart_pt_labels)


for i=1:2
    #freq = countmap(pt_labels)
    cmap = :rainbow
    f = Figure()
    #Label(f[1,1], "Valine Dipeptide Clustered with CATBOSS (SIMPLE) Segments")
    Label(f[1,1], "VDP Advanced DP Clustered with $(titles[i]) Segments")
    scatter(
        f[2, 1], 
        (360 .* X) .- 180, 
        color=point_labels[i], 
        colormap=cmap, 
        markersize = 1, 
        #fxaa = true, 
        #depthsorting=true, 
        #transparency=true,
    )
f
save("$(titles[i])-vdp.png", f)
end


In [ ]:
function draw_decision_graph(D; q=0.02, seg_lengths=nothing)
    N, _ = size(D)
    dc = quantile!([D[i,j] for i=1:N, j=1:N if i > j], q)
    if isnothing(seg_lengths)
        ρ = hcat(collect(1:N), [length(row[row .<= dc]) for row in eachrow(D)])
    else
        ρ = hcat(collect(1:N), [sum(seg_lengths .* exp.(-(D[i,:] / dc).^2)) for i in 1:N])
    end
    δ = zeros(N)
    nearest_denser_neighbor = zeros(N)
    for i=1:N
        indices = convert(Array{Int32},filter!(idx -> idx ∉ [i], ρ[ρ[:,2] .>= ρ[i, 2],:][:,1]))
        if length(indices) > 0
            δ[i] = minimum(D[indices, i])
            nearest_denser_neighbor[i] = argmin(D[indices, i])
        else
            δ[i] = maximum(D[:,i])
            nearest_denser_neighbor[i] = i
        end
    end
    γ = ρ[:,2] .* δ
    fig = Figure(size=(1500, 500))
    ax1 = Axis(fig[1,1], title="ρ-δ Plot", xlabel="ρ", ylabel="δ")
    ax2 = Axis(fig[1,2], title="γ Plot", xlabel="Rank", ylabel="γ")
    ax3 = Axis(fig[1,3], title="log γ Plot", xlabel="Rank", ylabel="γ")

    markersize = 4
    scatter!(ax1, ρ[:,2], δ, markersize=markersize)
    scatter!(ax2, 1:length(γ), sort!(γ), markersize=markersize)
    scatter!(ax3, 1:length(γ), log.(sort!(γ)), markersize=markersize)
    fig
end


segment_lengths = [jd_cps[T+1] - jd_cps[T] for T in 1:length(jd_cps) - 1]
draw_decision_graph(D, seg_lengths=segment_lengths)

In [ ]:
f = Figure(size=(1000,1000))
hist!(Axis(f[1,1],xticks=0:2:120,xticklabelrotation=45.0),seg_labels[:,1],bins=120)

f

In [ ]:
x1_cps_bart = convert(Array{Int32}, readdlm("../data/VDP/vdp_cps_x1.txt")) .+ 1
x2_cps_bart = convert(Array{Int32}, readdlm("../data/VDP/vdp_cps_x2.txt")) .+ 1
x3_cps_bart = convert(Array{Int32}, readdlm("../data/VDP/vdp_cps_x3.txt")) .+ 1

function lazy_cluster_1d(data, cps, N)
    c(x,y) = peuclidean(x,y,1.0).^2
    dists = pairwise_segment_distances_1d(data, cps, c)
    labels = get_clusters(data, cps, dists, N)
    pt_labels = label_series(data, cps, labels)
    return pt_labels
end


x1_cps_bart = x1_cps_bart[1:findfirst(x -> x > T, x1_cps_bart)[1] - 1]
x2_cps_bart = x2_cps_bart[1:findfirst(x -> x > T, x2_cps_bart)[1] - 1]
x3_cps_bart = x3_cps_bart[1:findfirst(x -> x > T, x3_cps_bart)[1] - 1]
x1_labels_bart = lazy_cluster_1d(X[:,1], x1_cps_bart, N[1])
x2_labels_bart = lazy_cluster_1d(X[:,2], x2_cps_bart, N[2])
x3_labels_bart = lazy_cluster_1d(X[:,3], x3_cps_bart, N[3])

In [ ]:
fig_size = (2000,1200)
fig = Figure(size=fig_size)
cmap=Makie.Categorical(:jet1)

ax1 = Axis(fig[1,1])
ax2 = Axis(fig[2,1])
ax3 = Axis(fig[3,1])
ax1.title="Valine Diepeptide Angle φ Segmented via Megafauna"
ax2.title="Valine Diepeptide Angle ψ Segmented via Megafauna"
ax3.title="Valine Diepeptide Angle χ Segmented via Megafauna"

ax1.xlabel=ax2.xlabel=ax3.xlabel="t"
ax1.ylabel=ax2.ylabel="x(t)"
ax1.xticks=ax2.xticks=ax3.xticks=0:(T ÷ 25):T
ax1.xtickformat=ax2.xtickformat=ax3.xtickformat="{:.0f}"
ax1.yticks=ax2.yticks=0:0.1:1

vlines!(ax1,x1_cps[:,1], linestyle=:dashdot, linewidth=1)
vlines!(ax2,x2_cps[:,1], linestyle=:dashdot, linewidth=1)
vlines!(ax3,x3_cps[:,1], linestyle=:dashdot, linewidth=1)

scatter!(ax1, 1:T, X[:,1], color=x1_labels, colormap=cmap, markersize=2)
scatter!(ax2, 1:T, X[:,2], color=x2_labels, colormap=cmap, markersize=2)
scatter!(ax3, 1:T, X[:,3], color=x3_labels, colormap=cmap, markersize=2)

current_figure()

In [ ]:
save("valine-anglewise-mf.png", fig)

In [ ]:
Y = cat(x1_labels, x2_labels, x3_labels, dims=2)
dummy_vec = [10^i for i in range(2, 0; step=-1)]'
series_labels = [dummy_vec * Y[t,:] for t in 1:T]
unique_labels = unique(series_labels)
labels = [findfirst(idx -> idx == label, unique_labels) for label in series_labels];

f = Figure()
Label(f[1,1], "Valine Dipeptide Clustered Componentwise")
scatter(f[2, 1], X, color=labels, colormap=cmap, markersize = 2, fxaa = true, depthsorting=true, transparency=true)
f

In [ ]:
save("valine-joint-clustering.png", f)

In [ ]:
d = 3
cps = x3_cps
segment_lengths = [cps[T+1] - cps[T] for T in 1:length(cps) - 1]
idx = sortperm(segment_lengths, rev=true);
fig = Figure(size=(1500, 1500))
for i = 1:2
T = idx[i]
segment = X[cps[T]:cps[T+1],d]
U = fit_mle(Normal, segment)
V = fit_mle(Laplace, segment)
W = fit_mle(Uniform, segment)
Uks = ExactOneSampleKSTest(segment, U)
Vks = ExactOneSampleKSTest(segment, V)
t = collect(range(minimum(segment), maximum(segment), length=1000))
ax = Axis(fig[i,1], title="Kolmogorov-Smirnov Test for segment #$(T)\n# Samples: $(length(segment))\nHypothesis: Laplace Distribution\np-value: $(round(pvalue(Vks),digits=3))", ylabel="CDF")
ax2 = Axis(fig[i,2], title="Kolmogorov-Smirnov Test for segment #$(T)\n# Samples: $(length(segment))\nHypothesis: Normal Distribution\np-value: $(round(pvalue(Uks),digits=3))", ylabel="CDF")
lines!(ax, t, cdf.(V,t), color="blue")
lines!(ax, sort(segment), (1:length(segment))./length(segment),color="red")
lines!(ax2, t, cdf.(U,t), color="blue")
lines!(ax2, sort(segment), (1:length(segment))./length(segment),color="red")
end
current_figure()

In [ ]:
d = 1
angles = ["φ", "ψ", "χ"]
cps = x1_cps
segment_lengths = [cps[T+1] - cps[T] for T in 1:length(cps) - 1]
idx = sortperm(segment_lengths, rev=true);
fig = Figure(size=(1500, 1500))
for i = 1:2
T = idx[i]
segment = X[x1_cps[T]:x1_cps[T+1],d]
U = fit_mle(Normal, segment)
V = fit_mle(Laplace, segment)
Uks = ExactOneSampleKSTest(segment, U)
Vks = ExactOneSampleKSTest(segment, V)
t = collect(range(minimum(segment), maximum(segment), length=1000))
ax = Axis(fig[i,1], title="Kolmogorov-Smirnov Test for angle $(angles[d]), segment #$(T)\n# Samples: $(length(segment))\nHypothesis: Laplace Distribution\np-value: $(round(pvalue(Vks),digits=3))", ylabel="CDF")
ax2 = Axis(fig[i,2], title="Kolmogorov-Smirnov Test for angle $(angles[d]), segment #$(T)\n# Samples: $(length(segment))\nHypothesis: Normal Distribution\np-value: $(round(pvalue(Uks),digits=3))", ylabel="CDF")
lines!(ax, t, cdf.(V,t), color="blue")
lines!(ax, sort(segment), (1:length(segment))./length(segment),color="red")
lines!(ax2, t, cdf.(U,t), color="blue")
lines!(ax2, sort(segment), (1:length(segment))./length(segment),color="red")

elem_1 = [LineElement(color = :red, linestyle = nothing)]
elem_2 = [LineElement(color = :blue, linestyle = nothing)]

if i == 1
    axislegend(ax,
        [elem_1, elem_2],
        ["Empirical CDF", "Hypothesis CDF"],
        patchsize = (5, 5), rowgap = 1, position=:lt)
    end
end
current_figure()

save("valine-x$(d)-KStest.png", fig)

In [ ]:
function draw_decision_graph(D; q=0.02, seg_lengths=nothing)
    N, _ = size(D)
    dc = quantile!([D[i,j] for i=1:N, j=1:N if i > j], q)
    if isnothing(seg_lengths)
        ρ = hcat(collect(1:N), [length(row[row .<= dc]) for row in eachrow(D)])
    else
        ρ = hcat(collect(1:N), [sum(seg_lengths .* exp.(-(D[i,:] / dc).^2)) for i in 1:N])
    end
    δ = zeros(N)
    nearest_denser_neighbor = zeros(N)
    for i=1:N
        indices = convert(Array{Int32},filter!(idx -> idx ∉ [i], ρ[ρ[:,2] .>= ρ[i, 2],:][:,1]))
        if length(indices) > 0
            δ[i] = minimum(D[indices, i])
            nearest_denser_neighbor[i] = argmin(D[indices, i])
        else
            δ[i] = maximum(D[:,i])
            nearest_denser_neighbor[i] = i
        end
    end
    γ = ρ[:,2] .* δ
    #fig = Figure(size=(1000, 500))
    #ax1 = Axis(fig[1,1], title="ρ-δ Plot", xlabel="ρ", ylabel="δ")
    #ax2 = Axis(fig[1,2], title="γ Plot", xlabel="Rank", ylabel="γ")
    #scatter!(ax1, ρ[:,2], δ)
    #scatter!(ax2, 1:length(γ), sort!(γ))
    return γ
end

c(x,y) = peuclidean(x,y,1.0).^2
x1_lengths = [x1_cps[T+1] - x1_cps[T] for T in 1:length(x1_cps) - 1]
x2_lengths = [x2_cps[T+1] - x2_cps[T] for T in 1:length(x2_cps) - 1]
x3_lengths = [x3_cps[T+1] - x3_cps[T] for T in 1:length(x3_cps) - 1]


fig=Figure(size=(1500,500))

ax1 = Axis(fig[1,1])
ax2 = Axis(fig[1,2])
ax3 = Axis(fig[1,3])
ax1.title="Valine Diepeptide Angle φ Decision Graph"
ax2.title="Valine Diepeptide Angle ψ Decision Graph"
ax3.title="Valine Diepeptide Angle χ Decision Graph"
ax1.xlabel=ax2.xlabel=ax3.xlabel="Rank"
ax1.ylabel=ax2.ylabel=ax3.ylabel="γ"
γ1 = draw_decision_graph(pairwise_segment_distances_1d(X[:,1], x1_cps, c), seg_lengths=x1_lengths)
γ2 = draw_decision_graph(pairwise_segment_distances_1d(X[:,2], x2_cps, c), seg_lengths=x2_lengths)
γ3 = draw_decision_graph(pairwise_segment_distances_1d(X[:,3], x3_cps, c), seg_lengths=x3_lengths)

scatter!(ax1, 1:length(γ1), sort!(γ1))
scatter!(ax2, 1:length(γ2), sort!(γ2))
scatter!(ax3, 1:length(γ3), sort!(γ3))
current_figure()


In [ ]:
save("valine-decision-graphs.png", fig)

In [ ]:
function identify_transitions(X, cps)
    N = length(cps) - 1
    t1, t2, t3, t4 = cps[1], cps[2], cps[3], cps[4]
    predecessor = X[t1:t2]
    current = X[t2:t3]
    successor = X[t3:t4]
    m_pred, σ_pred = median(predecessor), std(predecessor)
    m_curr, σ_curr = median(current), std(current)
    m_succ, σ_succ = median(successor), std(successor)
    transition_labels = zeros(N)
    mstable_stats = zeros(N)
    trans_stats = zeros(N)
    for i = 2:N-1
        metalike = 0
        translike = 0
        L = length(current)
        for (idx, x) in enumerate(current)
            λ = idx / (L + 1)
            μ = λ * median(successor) + (1 - λ) * median(predecessor)
            ρ = λ * std(successor) + (1 - λ) * std(predecessor)
            metalike -= log(2* σ_curr) + (abs(x - m_curr) / σ_curr)
            translike -= log(2 * ρ) + (abs(x - μ) / ρ)
        end
        mstable_stats[i] = metalike 
        trans_stats[i] = translike
        transition_labels[i] = metalike > translike ? 0 : 1
        if i == N - 1
            break
        end
        predecessor = current
        current = successor
        successor = X[cps[i+2]:cps[i+3]]
        m_pred, σ_pred = m_curr, σ_curr
        m_curr, σ_curr = m_succ, σ_succ
        m_succ, σ_succ = median(successor), std(successor)
    end
    return (transition_labels, mstable_stats, trans_stats)
end

x1_segment_transition_labels, x1_segment_mstable_stats, x1_segment_trans_stats = identify_transitions(X[:,1], x1_cps);
x2_segment_transition_labels, x2_segment_mstable_stats, x2_segment_trans_stats = identify_transitions(X[:,2], x2_cps);
x3_segment_transition_labels, x3_segment_mstable_stats, x3_segment_trans_stats = identify_transitions(X[:,3], x3_cps);

In [ ]:
    
N = length(x3_cps) - 1
laplace_pvalues = []
normal_pvalues = []
mstable_lengths = []
n_mstable = 0

for i=1:N
    if x3_segment_transition_labels[i] > 0
        continue
    end
    n_mstable += 1
    segment = X[x3_cps[i]:x3_cps[i+1],3]
    push!(mstable_lengths, length(segment))
    Z = fit_mle(Laplace, segment)
    Zprime = fit_mle(Normal, segment)
    push!(laplace_pvalues, pvalue(ExactOneSampleKSTest(segment, Z)))
    push!(normal_pvalues, pvalue(ExactOneSampleKSTest(segment, Zprime)))
end

println("Number of metastable segments: $n_mstable")
println("Percentage of metastable segments that reject the Laplace hypothesis 
    $(1 - count(>(0.05), laplace_pvalues) / n_mstable))")
println("Median length of metsatsble segments $(median(mstable_lengths))")
println("IQR of length of metastable segments $(quantile(mstable_lengths, 0.75) - quantile(mstable_lengths, 0.25))")
println("Percentage of metastable segments that reject the Normal hypothesis 
    $(1 - (count(>(0.05), normal_pvalues)) / n_mstable )")
